## Full Attack 모델 테스트

### **0. 필요한 라이브러리 import 및 Data 불러오기**

In [1]:
import classification_util
import pandas as pd

files={
    'CICI':'/home/irteam/dcloud-global-dir/MLAC/new_data/CICI.csv',
    'UNSW':'/home/irteam/dcloud-global-dir/MLAC/new_data/UNSW.csv'
}

CICI_data = pd.read_csv(files['CICI'])
UNSW_data = pd.read_csv(files['UNSW'])

In [2]:
models = classification_util.getModels()

### **1. 데이터 분류하기**

In [3]:
attack_23={0:'Benign', 1: 'Analysis', 2: 'Backdoor', 3: 'Bot', 4: 'DDoS', 5: 'DoS', 6: 'DoS GoldenEye', 7: 'DoS Slowhttptest', 8: 'DoS slowloris', 9: 'Dos Hulk', 10: 'Exploits', 11: 'FTP-Patator', 12: 'Fuzzers', 13: 'Generic', 14: 'Heartbleed', 15: 'Infiltration', 16: 'PortScan', 17: 'Reconnaissance', 18: 'SSH-Patator', 19: 'Shellcode', 20: 'Web Attack – Brute Force', 21: 'Web Attack – Sql Injection', 22: 'Web Attack – XSS', 23: 'Worms'}

In [4]:
class_1_CICI_data=CICI_data[CICI_data['nist_category']==1]
class_2_CICI_data=CICI_data[CICI_data['nist_category']==2]
class_3_CICI_data=CICI_data[CICI_data['nist_category']==3]
class_4_CICI_data=CICI_data[CICI_data['nist_category']==4]

In [ ]:
class_1_UNSW_data=UNSW_data[UNSW_data['nist_category']==1]
class_2_UNSW_data=UNSW_data[UNSW_data['nist_category']==2]
class_3_UNSW_data=UNSW_data[UNSW_data['nist_category']==3]
class_4_UNSW_data=UNSW_data[UNSW_data['nist_category']==4]

In [5]:
class_1_CICI_data['attack_category'].value_counts()

16    158930
17     15203
20      1507
22       652
21        21
14        11
Name: attack_category, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
def TrainTestSplit(type, data):
    print('Dataset Split')
    if type == 'B':
        print('Binary Classification Dataset Split')
        target = data['label']
    elif type == 'N':
        print('NIST Standard Classification Dataset Split')
        # Remove Benign Data
        benign = data[data['label'] == 0].index
        data.drop(benign, inplace=True)
        target = data['nist_category']
    elif type == 'F':
        print('Full attack Classification Dataset Split')
        target = data['attack_category']

    del data['attack_category']
    del data['label']
    del data['nist_category']
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, shuffle=True, stratify=target, random_state=34)
    return X_train, X_test, y_train, y_test

In [7]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [8]:
"""
Plot Confusion Matrix
"""
# confusion matrix plot
def plot_confusion_matrix(con_mat,labels,title:str,cmap=plt.cm.get_cmap('Blues'),normalize=False, confusion_path=None):
    plt.imshow(con_mat,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks=np.arange(len(labels))
    nlabels=[]
    for k in range(len(con_mat)):
        n=sum(con_mat[k])
        nlabel='{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)

    plt.xticks(marks,labels,rotation=45)
    plt.yticks(marks,nlabels)

    thresh=con_mat.max()/2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.show()
    #이미지 저장
    plt.savefig(confusion_path+'/'+title+'.png',facecolor='#eeeeee',edgecolor='blue',pad_inches=0.5)
    plt.clf()

In [11]:
def MultiClassification(dtype, type, file, models, X_train, X_test, y_train, y_test):
    accuracy = pd.DataFrame(columns=['Model','Acc','F1_mi','Recall_mi','Precision_mi','F1_ma','Recall_ma','Precision_ma','F1_we','Recall_we','Precision_we','Execution'])
    print('Model\tAcc\tF1_mi\tRecall_mi\tPrecision_mi\tF1_ma\tRecall_ma\tPrecision_ma\tF1_we\tRecall_we\tPrecision_we\tExecution')
    cnt = 0
    for name, model in models:
        start_time = time.time()
        # 모델 훈련 및 예측
        model.fit(X_train, y_train)
        prediction = model.predict(X_test)
        # 지표 추출
        delta = time.time() - start_time
        acc = accuracy_score(y_test, prediction)
        f1_mi = f1_score(y_test, prediction,average='micro')
        f1_ma = f1_score(y_test, prediction,average='macro')
        f1_we = f1_score(y_test, prediction,average='weighted')
        recall_mi = recall_score(y_test, prediction, average='micro')
        recall_ma = recall_score(y_test, prediction, average='macro')
        recall_we = recall_score(y_test, prediction, average='weighted')
        precision_mi = precision_score(y_test, prediction, average='micro')
        precision_ma = precision_score(y_test, prediction, average='macro')
        precision_we = precision_score(y_test, prediction, average='weighted')
        confusion = metrics.confusion_matrix(y_test, prediction)
        # 저장
        accuracy.loc[cnt] = [name, acc, f1_mi, f1_ma, f1_we, recall_mi, recall_ma, recall_we, precision_mi, precision_ma, precision_we, delta]
        print('{}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.2f} secs'.format(name, acc, f1_mi, f1_ma, f1_we, recall_mi, recall_ma, recall_we, precision_mi, precision_ma, precision_we, delta))
        cnt += 1
        marks = np.arange(len(y_train.value_counts()))
        if type == 'N':
            if dtype == 'C':
                os.makedirs(os.getcwd()+'/Matrix/NIST/CICI',exist_ok=True)
                plot_confusion_matrix(confusion,labels=marks,title=name,confusion_path=os.getcwd()+'/Matrix/NIST/CICI')
            elif dtype == 'U':
                os.makedirs(os.getcwd()+'/Matrix/NIST/UNSW',exist_ok=True)
                plot_confusion_matrix(confusion,labels=marks,title=name,confusion_path=os.getcwd()+'/Matrix/NIST/UNSW')     
                
        if type == 'F':
            if dtype == 'C':
                os.makedirs(os.getcwd()+'/Matrix/FULL/CICI/3',exist_ok=True)
                plot_confusion_matrix(confusion,labels=marks,title=name,confusion_path=os.getcwd()+'/Matrix/FULL/CICI/3')
            elif dtype == 'U':
                os.makedirs(os.getcwd()+'/Matrix/FULL/UNSW/3',exist_ok=True)
                plot_confusion_matrix(confusion,labels=marks,title=name,confusion_path=os.getcwd()+'/Matrix/FULL/UNSW/3') 
                
    accuracy = accuracy.round(3)
    accuracy.to_csv(os.getcwd()+'/Score/'+file+'.csv',index=False)

### !!! 파일 저장 위치 바뀔 수 있도록 위에서 설정해줘야함!

In [43]:
import classification_util
class_1_CICI_X_train, class_1_CICI_X_test, class_1_CICI_y_train, class_1_CICI_y_test =TrainTestSplit('F', class_1_CICI_data)
MultiClassification(dtype='C', type='F', file='class_1_CICI', models=models, X_train= class_1_CICI_X_train, X_test= class_1_CICI_X_test, y_train= class_1_CICI_y_train, y_test= class_1_CICI_y_test)

Dataset Split
Full attack Classification Dataset Split
Model	Acc	F1_mi	Recall_mi	Precision_mi	F1_ma	Recall_ma	Precision_ma	F1_we	Recall_we	Precision_we	Execution
RF	0.995	0.995	0.724	0.994	0.995	0.709	0.995	0.995	0.928	0.996	0.17 secs
DT	0.996	0.996	0.768	0.995	0.996	0.759	0.996	0.996	0.927	0.997	0.33 secs
NB	0.987	0.987	0.599	0.988	0.987	0.819	0.987	0.987	0.569	0.989	0.12 secs
LDA	0.993	0.993	0.710	0.992	0.993	0.826	0.993	0.993	0.765	0.997	0.62 secs
QDA	0.993	0.993	0.561	0.992	0.993	0.609	0.993	0.993	0.644	0.997	0.21 secs
LR	0.996	0.996	0.644	0.995	0.996	0.671	0.996	0.996	0.734	0.996	13.01 secs
ABoost	0.928	0.928	0.435	0.908	0.928	0.402	0.928	0.928	0.593	0.926	7.76 secs
k-NN	0.996	0.996	0.792	0.996	0.996	0.775	0.996	0.996	0.820	0.996	52.72 secs
MLP	0.996	0.996	0.647	0.995	0.996	0.672	0.996	0.996	0.732	0.996	8.43 secs
SVM	0.996	0.996	0.793	0.995	0.996	0.789	0.996	0.996	0.930	0.997	3.02 secs


<Figure size 432x288 with 0 Axes>

In [10]:
class_2_CICI_X_train, class_2_CICI_X_test, class_2_CICI_y_train, class_2_CICI_y_test =TrainTestSplit('F', class_2_CICI_data)
MultiClassification(dtype='C', type='F', file='class_2_CICI', models=models, X_train= class_2_CICI_X_train, X_test= class_2_CICI_X_test, y_train= class_2_CICI_y_train, y_test= class_2_CICI_y_test)

Dataset Split
Full attack Classification Dataset Split
Model	Acc	F1_mi	Recall_mi	Precision_mi	F1_ma	Recall_ma	Precision_ma	F1_we	Recall_we	Precision_we	Execution
RF	0.787	0.787	0.530	0.743	0.787	0.512	0.787	0.787	0.620	0.769	0.20 secs
DT	0.773	0.773	0.552	0.744	0.773	0.551	0.773	0.773	0.559	0.730	0.74 secs
NB	0.209	0.209	0.437	0.193	0.209	0.677	0.209	0.209	0.576	0.728	0.09 secs
LDA	0.777	0.777	0.602	0.748	0.777	0.620	0.777	0.777	0.641	0.759	0.38 secs
QDA	0.395	0.395	0.480	0.469	0.395	0.625	0.395	0.395	0.589	0.876	0.17 secs
LR	0.792	0.792	0.562	0.755	0.792	0.548	0.792	0.792	0.737	0.800	9.11 secs
ABoost	0.605	0.605	0.210	0.467	0.605	0.228	0.605	0.605	0.208	0.383	10.58 secs
k-NN	0.808	0.808	0.593	0.779	0.808	0.571	0.808	0.808	0.674	0.796	17.00 secs
MLP	0.812	0.812	0.604	0.780	0.812	0.583	0.812	0.812	0.788	0.823	45.31 secs
SVM	0.794	0.794	0.568	0.757	0.794	0.549	0.794	0.794	0.652	0.765	27.16 secs


<Figure size 432x288 with 0 Axes>

In [12]:
class_3_CICI_X_train, class_3_CICI_X_test, class_3_CICI_y_train, class_3_CICI_y_test =TrainTestSplit('F', class_3_CICI_data)
MultiClassification(dtype='C', type='F', file='class_3_CICI', models=models, X_train= class_3_CICI_X_train, X_test= class_3_CICI_X_test, y_train= class_3_CICI_y_train, y_test= class_3_CICI_y_test)

Dataset Split
Full attack Classification Dataset Split
Model	Acc	F1_mi	Recall_mi	Precision_mi	F1_ma	Recall_ma	Precision_ma	F1_we	Recall_we	Precision_we	Execution
RF	0.991	0.991	0.945	0.990	0.991	0.915	0.991	0.991	0.983	0.991	0.90 secs
DT	0.972	0.972	0.800	0.970	0.972	0.760	0.972	0.972	0.872	0.971	3.62 secs
NB	0.692	0.692	0.628	0.773	0.692	0.772	0.692	0.692	0.679	0.959	0.26 secs
LDA	0.994	0.994	0.955	0.994	0.994	0.962	0.994	0.994	0.949	0.994	1.43 secs
QDA	0.982	0.982	0.952	0.984	0.982	0.991	0.982	0.982	0.930	0.989	0.49 secs
LR	0.998	0.998	0.984	0.998	0.998	0.984	0.998	0.998	0.984	0.998	30.70 secs
ABoost	0.936	0.936	0.570	0.927	0.936	0.628	0.936	0.936	0.556	0.923	51.82 secs
